# **INTERVIEW QUESTION**
## 1 question
**1. Find out cummulative sales or running total sales**

**2. find out the prev sales**

**3. find out the nex sales**

#### SOLVE USING PYSPARK AND SPARK SQL

In [0]:
data = [ ['2024-01-01',20000], ['2024-01-02',10000],[ '2024-01-03',150000], ['2024-01-04',100000], ['2024-01-05',210000]] 
  
#define column names
columns = ['date', 'sales'] 
  
#create dataframe using data and column names
df = spark.createDataFrame(data,columns)
display(df)

date,sales
2024-01-01,20000
2024-01-02,10000
2024-01-03,150000
2024-01-04,100000
2024-01-05,210000


In [0]:
df.createOrReplaceTempView('sample')

In [0]:
%sql
select * from sample

date,sales
2024-01-01,20000
2024-01-02,10000
2024-01-03,150000
2024-01-04,100000
2024-01-05,210000


In [0]:
%sql
select *,sum(sales) over(order by date asc) cumm_sales from sample

date,sales,cumm_sales
2024-01-01,20000,20000
2024-01-02,10000,30000
2024-01-03,150000,180000
2024-01-04,100000,280000
2024-01-05,210000,490000


In [0]:
%sql
select *,sum(sales) over(order by date asc) cumm_sales, lag(sales) over(order by date asc) prev_sales  from sample

date,sales,cumm_sales,prev_sales
2024-01-01,20000,20000,null
2024-01-02,10000,30000,20000
2024-01-03,150000,180000,10000
2024-01-04,100000,280000,150000
2024-01-05,210000,490000,100000


In [0]:
%sql
select *,sum(sales) over(order by date asc) cumm_sales, lag(sales) over(order by date asc) prev_sales , lead(sales) over(order by date asc) Next_sales from sample

date,sales,cumm_sales,prev_sales,Next_sales
2024-01-01,20000,20000,null,10000
2024-01-02,10000,30000,20000,150000
2024-01-03,150000,180000,10000,100000
2024-01-04,100000,280000,150000,210000
2024-01-05,210000,490000,100000,null


In [0]:
display(df)

date,sales
2024-01-01,20000
2024-01-02,10000
2024-01-03,150000
2024-01-04,100000
2024-01-05,210000


In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

window = Window.orderBy('date')
df.withColumn('cumm_sales',sum(col('sales')).over(window)).show()

+----------+------+----------+
|      date| sales|cumm_sales|
+----------+------+----------+
|2024-01-01| 20000|     20000|
|2024-01-02| 10000|     30000|
|2024-01-03|150000|    180000|
|2024-01-04|100000|    280000|
|2024-01-05|210000|    490000|
+----------+------+----------+



In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

window = Window.orderBy('date')
df1=df.withColumn('cumm_sales',sum(col('sales')).over(window))
df1.withColumn('prev_sales',lag(col('sales')).over(window)).show()

+----------+------+----------+----------+
|      date| sales|cumm_sales|prev_sales|
+----------+------+----------+----------+
|2024-01-01| 20000|     20000|      null|
|2024-01-02| 10000|     30000|     20000|
|2024-01-03|150000|    180000|     10000|
|2024-01-04|100000|    280000|    150000|
|2024-01-05|210000|    490000|    100000|
+----------+------+----------+----------+



In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

window = Window.orderBy('date')
df1=df.withColumn('cumm_sales',sum(col('sales')).over(window))
df2=df1.withColumn('prev_sales',lag(col('sales')).over(window))
df2.withColumn('next_sales',lead(col('sales')).over(window)).show()

+----------+------+----------+----------+----------+
|      date| sales|cumm_sales|prev_sales|next_sales|
+----------+------+----------+----------+----------+
|2024-01-01| 20000|     20000|      null|     10000|
|2024-01-02| 10000|     30000|     20000|    150000|
|2024-01-03|150000|    180000|     10000|    100000|
|2024-01-04|100000|    280000|    150000|    210000|
|2024-01-05|210000|    490000|    100000|      null|
+----------+------+----------+----------+----------+

